In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
def graficar(grafico, titulo, x_label, y_label):
    grafico.set_title(titulo)
    grafico.set_xlabel(x_label)
    grafico.set_ylabel(y_label)
    return grafico


In [ ]:
diccionario_tipos = {
    "tipodepropiedad":"category",
    "provincia":"category",
    "provincia":"category",
    #"habitaciones":"int",
    #"garages":"int",
    #"banos":"int",
    #'metroscubiertos':, 'metrostotales', 'idzona', 'lat', 'lng',
    'gimnasio':"bool",
    'usosmultiples':"bool",
    'piscina':"bool",
    'escuelascercanas':"bool",
    'centroscomercialescercanos':"bool"
}
df = pd.read_csv("data/train.csv", dtype = diccionario_tipos)

In [ ]:
df[["antiguedad"]].describe()

In [ ]:
g = df["antiguedad"].plot.hist(bins=16)
g.set_title("Histograma de la antiguedad de las propiedades")
g.set_xlabel("Antiguedad")
g.set_ylabel("Frecuencia")

Podemos ver como la distribucion de la antiguedad, nos da con mucha mayor frecuencia ventas de propiedades nuevas o casi nuevas, mientras que hay muchas menos propiedades mas antiguas

In [ ]:
cant_propiedades_nuevas = df.loc[df["antiguedad"]==0, "antiguedad"].count()
cant_propiedades_totales = df["antiguedad"].count()
print(f"La cantidad de propiedades nuevas es: {cant_propiedades_nuevas}, lo cual representa un {cant_propiedades_nuevas*100 / cant_propiedades_totales}%")

Como vemos las propiedades nuevas representan un **25%** de las propiedades en venta.
Por lo tanto vamos a ver cual es la distribución de la  antiguedad quitando las propiedades nuevas

In [ ]:
ventas_no_nuevas = df.loc[df["antiguedad"]>0]
ventas_no_nuevas["antiguedad"].plot.hist(bins = 32)

Podemos ver que aún sacando las ventas de inmuebles nuevos predominan las ventas de propiedades con una antiguedad **menor a diez**

## Analsisi de inmuebles nuevos
Como vimos, tenemos un cuarto de los inmuebles en venta que son a nuevo. Por lo tanto es interesante indagar sobre que características tienen estas propiedades: ¿Son mayoritariamente departamentos (porque se hacen edificios a nuevo)? ¿Tienen un precio mayor que la media? ¿Predominan en las provincias y ciudades mas importantes?

In [ ]:
nuevos = df.loc[df["antiguedad"] == 0]

#### Que tipos de propiedad predominan en las ventas a nuevo?

In [ ]:
nuevos["tipodepropiedad"].value_counts().plot.bar()

Vemos que predominan las casas y apartamentos y pareciera que hay algunas categorías que no tienen ninguna publicación a nuevo

In [ ]:
nuevos["tipodepropiedad"].value_counts().loc[lambda x: x == 0]

#### Locación de las propiedades nuevas

In [ ]:
fig = plt.figure(figsize=(15,7))
nuevos["provincia"].value_counts().plot("bar")

Las primeras 4 provincias concentran una gran cantidad de las ventas de propiedades a nuevo

In [ ]:
fig = plt.figure(figsize=(17,7))
nuevos["cantidad"] = 1 
nuev_prov_tipo = nuevos.groupby(["provincia", "tipodepropiedad"]).agg({"cantidad":sum})
nuev_prov_tipo.head(20)

#### Precio de las propiedades nuevas

In [ ]:
fig= plt.figure(figsize=(15, 7))
ax = plt.axes()
buckets = 60
g = nuevos["precio"].plot(kind = 'hist', bins = buckets, alpha = 0.7)
h = ventas_no_nuevas["precio"].sample(n=nuevos["precio"].count(), random_state = 500).plot.hist(bins = buckets, alpha = 0.4,)

fig.legend(["Precio nuevos", "Precio NO nuevos"])
#fig.add_subplot(h)
plt.title("Distribucion de los precios de la propiedades Nuevas vs No nuevas")
plt.ticklabel_format(style="plain") #SACA la notación científica de los precios
ax.xaxis.set_major_locator(ticker.MultipleLocator(1000000)) 
#Decimos cada cuantos pesos mexicanos queremos que aparezca una marca en el grafico

Como podemos ver la distribucion de los precios de las propiedades nuevas vs las no nuevas (con antiguedad > 0) tienen ciertas características observables:
 - En el rango [0, 1000000] tenemos una mayor frecuencia de propiedades NO nuevas en venta
 - En el rango [1000000, 3000000] tenemos mas propiedades nuevas en venta
 - En los precios siguientes tienen distribuciones muy similares
 
Por lo tanto podría concluir que las propiedades nuevas se venden a un precio ligeramente mayor  
